In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'hack-sochi-train-original:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4792080%2F8112035%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240414%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240414T053520Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9de078a0f40dc73241a3b8128ca1458b85c9bb39d8feaa6684c0902a97110456e7c0ebd91d3d7c251d96e461523e283c38cae44733adbdbd45f94c3dc6e2f03afe7c7d128cc45d9f1fb31d765b9cacef2b633a32067678630e8cb0830f7e2dff55926f4ddad2c91923b87a0746ebfa96cc8b93fc2c39b2b3874488cda816235fde2b336f1204df5ad6b0950945f85caa5e6f62e308a4b22f1fe0394f955097fd8b002b6df71beba41d7f459dc749d061eb956cb104d1bc57765ca002961cc8d5f5fba43eece8660e8583abd000c8d15d880ddbee103877d1d3166e94656eb2bc52f4fdaed11151ad3987cea81bb40dfe6e82de28ec68db0b1251c3201f0117b3,yolov8n_hack_sochi/pytorch/yolov8n-v1-e100/12:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F25546%2F30836%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240414%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240414T053520Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dc287e47c257709853938d88f8a58ca2d8f3d0aabfa883893805a4c7cd24b122e13386d7ee93c07268d7426672c3b4cdb310fcc22372e643b1f05c7b73b149e49f23693a779f6fd5efbb198057387e54fb951889e999d032e0315acc250cfe478f04043d7e80e38ad31476312f8f8c7c5c7c82f97c5cc3c68d7b69f9685e81b22e015fcbc7a1a171a09fb5fbd6c7b6a10aa4948a4499860824b90aa8f7a9c10a5b2da3fe58aecc19c2c705280475be092e6e287c9dd09c19af2f6e58c697697dc34fcb028b122ddeeda87d79896e984f1fd3966800c4e8224849012131b0eebe2a210dd441d0080c47d800d994197ca9197bb74b9498a3a1395d81fac9b6174aa'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 454583382 bytes downloaded
Downloaded and uncompressed: hack-sochi-train-original
[==================================================] 11357513 bytes downloaded
Downloaded and uncompressed: yolov8n_hack_sochi/pytorch/yolov8n-v1-e100/12
Data source import complete.


In [2]:
%%capture
!pip install easyocr
!pip install ultralytics

In [3]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Загрузка данных
file_path = '/kaggle/input/hack-sochi-train-original/train.csv'  # Замените на ваш путь к CSV файлу
images_dir = '/kaggle/input/hack-sochi-train-original/train/train'  # Путь к изображениям

data = pd.read_csv(file_path, sep=';')


# Добавление полного пути к изображениям
data['image_path'] = data['Наименование файла'].apply(lambda x: os.path.join(images_dir, x) + '.jpg')

# Проверка на наличие пропущенных значений
print("Количество пропущенных значений в каждом столбце:")
print(data.isnull().sum())

# Удаление строк с пропущенными значениями
data_cleaned = data.dropna()

print("Данные очищены, если должны были быть.")


Количество пропущенных значений в каждом столбце:
Наименование файла     0
Наименование товара    0
Категория продукта     0
Цена                   0
image_path             0
dtype: int64
Данные очищены, если должны были быть.


In [4]:
# Перемешивание данных
data_cleaned = data_cleaned.sample(frac=1).reset_index(drop=True)

# Разделение данных на обучающую и валидационную выборки
train_size = int(0.87 * len(data_cleaned))  # 80% данных для обучения

train_data = data_cleaned[:train_size]
val_data = data_cleaned[train_size:]

print(f"Размер обучающей выборки: {len(train_data)}")
print(f"Размер валидационной выборки: {len(val_data)}")


Размер обучающей выборки: 200
Размер валидационной выборки: 31


**Часть 2. Загрузка YOLO**

In [5]:
import pandas as pd
import os
from PIL import Image
from torchvision import transforms
import torch
from ultralytics import YOLO

# Замените 'path_to_model' на путь к вашей сохраненной модели
model_path = '/kaggle/input/yolov8n_hack_sochi/pytorch/yolov8n-v1-e100/12/best(2).pt'
model = YOLO(model_path)
# model.eval()  # Перевод модели в режим оценки



**Часть 3. Прогонка данных**

In [6]:
image_path = train_data.iloc[11]['image_path']
image = Image.open(image_path).convert('RGB')
transform = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor()
])
input_tensor = transform(image).unsqueeze(0)

# Детекция текстовых областей
with torch.no_grad():
    outputs = model.predict(input_tensor, save=True)



0: 640x640 1 Cen, 2 Measures, 1 Name, 1 Price, 14.6ms
Speed: 1.4ms preprocess, 14.6ms inference, 988.2ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict


In [9]:
%%capture
# Функция для проверки наличия всех необходимых классов и их уникальности
def check_classes(boxes, required_classes={0, 3, 4}):
    # Извлечение классов с уверенностью выше 0.5
    detected_classes = [box.cls.item() for box in boxes if box.conf.item() > 0.5]
    class_counts = {cls: detected_classes.count(cls) for cls in detected_classes}
    # Проверка наличия каждого класса ровно один раз
    return all(class_counts.get(rc, 0) == 1 for rc in required_classes)


valid_images = []
indices_to_drop = []

for idx, row in data.iterrows():
    image_path = row['image_path']
    image = Image.open(image_path).convert('RGB')

    # Преобразование изображения
    input_tensor = transform(image).unsqueeze(0)  # использование той же трансформации, что и ранее

    # Детекция с помощью модели
    results = model(input_tensor)

    for result in results:
        boxes = result.boxes  # Boxes object for bounding box outputs
        classes = boxes.cls
        masks = result.masks  # Masks object for segmentation masks outputs
        keypoints = result.keypoints  # Keypoints object for pose outputs
        probs = result.probs  # Probs object for classification outputs
        result.show()  # display to screen


        # Проверка наличия всех требуемых классов
        if check_classes(result.boxes):
            valid_images.append(image_path)
        else:
            indices_to_drop.append(idx)

data_clean = data.drop(index=indices_to_drop)
data_clean.to_csv('cleaned_dataset.csv', index=False)


0: 640x640 1 Cen, 1 Measure, 1 Name, 1 Price, 7.7ms
Speed: 1.4ms preprocess, 7.7ms inference, 4.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 Cens, 1 Measure, 2 Names, 1 Price, 9.9ms
Speed: 1.4ms preprocess, 9.9ms inference, 4.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Cen, 1 Measure, 2 Names, 1 Price, 7.7ms
Speed: 1.3ms preprocess, 7.7ms inference, 4.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Cen, 1 Measure, 1 Name, 1 Price, 7.4ms
Speed: 1.3ms preprocess, 7.4ms inference, 4.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Cen, 1 Mesure-price, 1 Price, 7.7ms
Speed: 1.3ms preprocess, 7.7ms inference, 4.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Cen, 1 Measure, 1 Name, 1 Price, 9.1ms
Speed: 1.3ms preprocess, 9.1ms inference, 5.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Cen, 2 Measures, 1 Name, 1 Price, 7.4ms
Speed: 1.6ms preprocess, 7.4ms inference, 4.6ms post

In [17]:
print(result.boxes)

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0., 4., 3., 5., 1.], device='cuda:0')
conf: tensor([0.9622, 0.9555, 0.8227, 0.7631, 0.4367], device='cuda:0')
data: tensor([[3.6213e+02, 3.8481e+02, 5.6395e+02, 5.1992e+02, 9.6223e-01, 0.0000e+00],
        [4.4491e+02, 4.3490e+02, 5.4954e+02, 4.8939e+02, 9.5548e-01, 4.0000e+00],
        [3.6710e+02, 3.8398e+02, 5.2121e+02, 4.0866e+02, 8.2272e-01, 3.0000e+00],
        [3.6755e+02, 4.8577e+02, 5.3896e+02, 5.1759e+02, 7.6310e-01, 5.0000e+00],
        [4.9941e+02, 3.8467e+02, 5.1934e+02, 4.0716e+02, 4.3673e-01, 1.0000e+00]], device='cuda:0')
id: None
is_track: False
orig_shape: (640, 640)
shape: torch.Size([5, 6])
xywh: tensor([[463.0366, 452.3685, 201.8198, 135.1115],
        [497.2291, 462.1466, 104.6304,  54.4839],
        [444.1534, 396.3200, 154.1126,  24.6727],
        [453.2589, 501.6816, 171.4103,  31.8184],
        [509.3745, 395.9166,  19.9299,  22.4833]], device='cuda:0')
xywhn: tensor([[0.7235, 0.7068, 0.315

In [19]:
def crop_and_save_image(image_path, coords, cls_name, original_name):
    image = Image.open(image_path)

    img_width, img_height = image.size

    # Преобразование относительных координат в абсолютные
    x1, y1, x2, y2 = coords
    x1 = x1 * img_width
    y1 = y1 * img_height
    x2 = x2 * img_width
    y2 = y2 * img_height

    # Обрезка изображения по координатам
    cropped_image = image.crop((x1, y1, x2, y2))

    # Создание папки если не существует
    if not os.path.exists(cls_name):
        os.makedirs(cls_name)

    # Сохранение изображения
    new_name = f"{original_name}_{cls_name}.jpg"
    cropped_image.save(os.path.join(cls_name, new_name))

In [18]:
import shutil

shutil.rmtree('/content/Name')
shutil.rmtree('/content/Price')

In [20]:

for idx, row in data_clean.iterrows():
    image_path = row['image_path']
    image = Image.open(image_path)
    img_width, img_height = image.size
    name_data=[]
    price_data=[]

    input_tensor = transform(image.convert('RGB')).unsqueeze(0)

    with torch.no_grad():
        outputs = model.predict(input_tensor, save=True)

    for result in outputs:
      for box, classes, conf in zip(result.boxes.xyxyn.tolist(), result.boxes.cls.tolist(), result.boxes.conf.tolist()):
          if classes in (3, 4):
            if classes == 3 and conf > 0.5:  # Name class
                crop_and_save_image(image_path, box, 'Name', row['Наименование файла'])
                name_data.append({'file_name': f"{row['Наименование файла']}_Name.jpg", 'name': row['Наименование товара']})
            elif classes == 4 and conf > 0.5:  # Price class
                crop_and_save_image(image_path, box, 'Price', row['Наименование файла'])
                price_data.append({'file_name': f"{row['Наименование файла']}_Price.jpg", 'price': row['Цена']})

# Сохранение новых CSV файлов
pd.DataFrame(name_data).to_csv('Name/name_data.csv', index=False)
pd.DataFrame(price_data).to_csv('Price/price_data.csv', index=False)


0: 640x640 1 Cen, 1 Measure, 1 Name, 1 Price, 11.4ms
Speed: 1.6ms preprocess, 11.4ms inference, 7.1ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

0: 640x640 1 Cen, 1 Measure, 1 Name, 1 Price, 7.8ms
Speed: 1.3ms preprocess, 7.8ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

0: 640x640 1 Cen, 1 Measure, 1 Name, 1 Price, 9.4ms
Speed: 1.8ms preprocess, 9.4ms inference, 4.9ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

0: 640x640 1 Cen, 2 Measures, 1 Name, 1 Price, 8.3ms
Speed: 1.3ms preprocess, 8.3ms inference, 6.7ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

0: 640x640 2 Cens, 1 Measure, 2 Names, 1 Price, 8.5ms
Speed: 1.3ms preprocess, 8.5ms inference, 4.6ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

0: 640x640 1 Cen, 1 Measure, 1 Name, 1 Price, 10.0ms
Speed: 2.9m

In [ ]:
!pip install easyocr

In [21]:
import easyocr
import re

reader = easyocr.Reader(['ru'])

final_data = {}

def read_text(image_path):
    image = Image.open(image_path)
    result = reader.readtext(image)
    # Объединяем весь распознанный текст
    full_text = ' '.join([text[1] for text in result])
    return full_text

def extract_numbers(text):
    return ' '.join(re.findall(r'\d+', text))  # Извлекаем только числовые группы

for filename in os.listdir('/content/Name'):
    if filename.endswith(".jpg"):
        image_path = os.path.join('/content/Name', filename)
        name_text = read_text(image_path)
        original_name = filename.replace("_Name.jpg", "")
        final_data[original_name] = {'name': name_text}

# Обработка изображений из папки Price
for filename in os.listdir('/content/Price'):
    if filename.endswith(".jpg"):
        image_path = os.path.join('/content/Price', filename)
        price_text = read_text(image_path)
        price_numbers = extract_numbers(price_text)  # Извлекаем только числа
        original_name = filename.replace("_Price.jpg", "")
        if original_name in final_data:
            final_data[original_name]['price'] = price_numbers

# Сохраняем в CSV
df = pd.DataFrame([(k, v['name'], v.get('price', '')) for k, v in final_data.items()], columns=['original_name', 'name', 'price'])
df.to_csv('final_output.csv', index=False)

print("Финальный CSV файл создан с именами и ценами.")

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.1% CompleteФинальный CSV файл создан с именами и ценами.
